<a href="https://colab.research.google.com/github/UG-isLab/introduction/blob/main/src/introduction/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データセットの読み込み
## 圧縮ファイルを展開する

一度だけ

In [19]:
!pip install tqdm

In [20]:
# import tarfile
# from tqdm import tqdm #進捗状況を視覚化

# tar_path = '/content/drive/MyDrive/Dataset/train.tar.gz'
# extract_path = '/content/drive/MyDrive/Dataset/train'

# with tarfile.open(tar_path, 'r:gz') as tar:
#     members = tar.getmembers()
#     for member in tqdm(members, desc="Extracting files"):
#         tar.extract(member, path=extract_path)


In [21]:
# tar_path = '/content/drive/MyDrive/Dataset/test.tar.gz'
# extract_path = '/content/drive/MyDrive/Dataset/test'

# with tarfile.open(tar_path, 'r:gz') as tar:
#     members = tar.getmembers()
#     for member in tqdm(members, desc="Extracting files"):
#         tar.extract(member, path=extract_path)

In [22]:
# tar_path = '/content/drive/MyDrive/Dataset/val.tar.gz'
# extract_path = '/content/drive/MyDrive/Dataset/valid'

# with tarfile.open(tar_path, 'r:gz') as tar:
#     members = tar.getmembers()
#     for member in tqdm(members, desc="Extracting files"):
#         tar.extract(member, path=extract_path)


## データセットを成形

In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

train_dir = '/content/drive/MyDrive/Dataset/train/train'
valid_dir = '/content/drive/MyDrive/Dataset/valid/val'
test_dir = '/content/drive/MyDrive/Dataset/test/test'

画像サイズとバッチサイズの設定

In [24]:
img_size = (64, 64)
batch_size = 32

In [25]:
# 画像データセット読み込み
#サブフォルダの名前によって自動的にラベリングしてくれる
train_ds = keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True
)

Found 8980 files belonging to 2 classes.


In [26]:
# クラス数取得
num_classes = len(train_ds.class_names)
print(f"クラス名: {train_ds.class_names}")

クラス名: ['0', '1']


In [27]:
test_ds = keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True
)

Found 2458 files belonging to 2 classes.


In [28]:
print(test_ds.class_names)

['0', '1']


In [29]:
valid_ds = keras.utils.image_dataset_from_directory(
    valid_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True
)

Found 1448 files belonging to 2 classes.


In [30]:
print(valid_ds.class_names)

['0', '1']


In [31]:
AUTOTUNE = tf.data.AUTOTUNE

# 正規化
normalization_layer = layers.Rescaling(1./255)

# 特徴量を正規化してらべるはそのまま
# データ読み込みと学習を並列化
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)).prefetch(buffer_size=AUTOTUNE)
valid_ds = valid_ds.map(lambda x, y: (normalization_layer(x), y)).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y)).prefetch(buffer_size=AUTOTUNE)


In [32]:
# 活性化関数はレル関数
# フィルタサイズ32 カーネルサイズ3
#入力は64,64,3
#レイヤーは畳み込み層＋プーリング層を3セット＋全結合層2層
#最後にソフトマックス関数で出力
model = keras.Sequential([
    layers.Conv2D(32, 3, activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

#モデルのコンパイル
# 最適化方法はアダム
#損失関数はクロスエントロピー
#評価指数は正解率
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 683,458 (2.61 MB)

 Trainable params: 683,458 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

In [33]:
# アーリーストッピングを設定
#検証データの損失関数を監視
#3エポック連続で皆瀬しなかったら終了
#止まった場合、過去の最もよかったモデルにする
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# 学習
#エポック数20
history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=20,
    callbacks=[early_stopping]
)


Epoch 1/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 178s 618ms/step - accuracy: 0.8079 - loss: 0.4240 - val_accuracy: 0.8688 - val_loss: 0.3914
Epoch 2/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 105s 372ms/step - accuracy: 0.9466 - loss: 0.1783 - val_accuracy: 0.8819 - val_loss: 0.3281
Epoch 3/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 105s 373ms/step - accuracy: 0.9662 - loss: 0.1265 - val_accuracy: 0.8826 - val_loss: 0.3976
Epoch 4/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 103s 367ms/step - accuracy: 0.9677 - loss: 0.1228 - val_accuracy: 0.8743 - val_loss: 0.3280
Epoch 5/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 102s 364ms/step - accuracy: 0.9724 - loss: 0.0986 - val_accuracy: 0.8833 - val_loss: 0.3709
Epoch 6/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 137s 348ms/step - accuracy: 0.9720 - loss: 0.0908 - val_accuracy: 0.8826 - val_loss: 0.3592
Epoch 7/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 99s 350ms/step - accuracy: 0.9722 - loss: 0.0844 - val_accuracy: 0.8895 - val_loss: 0.3119
Epoch 8/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 102s 361ms/step - accuracy: 0.9734 - 

In [34]:
# 評価
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc:.4f}")


77/77 ━━━━━━━━━━━━━━━━━━━━ 13s 169ms/step - accuracy: 0.9272 - loss: 0.2206
Test Accuracy: 0.9276
